In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
import scipy.special as special
import bilby
import H1 as get_interf

In [ ]:
#### Define time of event in GPS
time_of_event=1126259462.4
#### Get the data and PSD
H1 = get_interf.get_H1(time_of_event=1126259462.4, post_trigger_duration=2, duration=4, psd_duration_multi=32)

In [ ]:
#### Our model
pc = (const.pc).value
Gpc = pow(10,9)*pc
M_sun = (const.M_sun).value
G = (const.G).value
c = (const.c).value
#### Defining the model
def echo(frequency_array, R, d, M, D, wr, wi, alpha, t_0, phi):

    
    w = 2 * np.pi * frequency_array
    
    
    Zp = np.sqrt(np.pi/2) * (M/D) * (M_sun/Gpc) * np.float64(G/(c**2)) * (np.exp(1j*(w-(1j*wi)/(M*(M_sun*(G/(c**3)))))*t_0)*(1+R)*special.gamma(1-((1j*w)/(alpha/(M*(M_sun*(G/(c**3)))))))*(((wr/(M*(M_sun*(G/(c**3)))))*np.sin(((wr/(M*(M_sun*(G/(c**3)))))*t_0)+phi)+((1j*(w-(1j*wi)/(M*(M_sun*(G/(c**3))))))*np.cos((wr/(M*(M_sun*(G/(c**3))))*t_0)+phi))))/
                                                                          ((((w-(1j*wi)/(M*(M_sun*(G/(c**3)))))*(w-(1j*wi)/(M*(M_sun*(G/(c**3))))))-((wr/(M*(M_sun*(G/(c**3)))))**2))*((np.pi*special.gamma(1-((1j*w)/(alpha/(M*(M_sun*(G/(c**3))))))))+(np.exp(2j*d*(M*(M_sun*(G/(c**3))))*w)*R*np.cosh((np.pi*wr)/alpha)*special.gamma((1/2)-(1j*(w+(wr/(M*(M_sun*(G/(c**3))))))/(alpha/(M*(M_sun*(G/(c**3)))))))*special.gamma((1/2)-(1j*(w-(wr/(M*(M_sun*(G/(c**3))))))/(alpha/(M*(M_sun*(G/(c**3)))))))*special.gamma(1+((1j*w)/(alpha/(M*(M_sun*(G/(c**3)))))))))))

    cross = np.zeros(len(frequency_array))
    return {"plus": Zp, "cross": cross}

In [ ]:
#### Define the Likelihood according to what bilby likes
class My_Likelihood(bilby.Likelihood):

    def __init__(self, interferometers, waveform_generator, priors=None):

        super(My_Likelihood, self).__init__(dict())
        self.interferometers = interferometers[0]
        self.waveform_generator = waveform_generator
        self.priors = priors

    def priors(self):
        return self.priors

    def log_likelihood(self):

        waveform = self.waveform_generator.frequency_domain_strain(self.parameters)
        residual = self.interferometers.frequency_domain_strain - \
                    self.interferometers.get_detector_response(waveform, self.parameters)
        psd = self.interferometers.power_spectral_density_array
        duration = self.waveform_generator.duration

        log_l = -2.0 / duration * np.sum((np.conj(residual)*residual) / psd)
        
        return log_l.real

    def noise_log_likelihood(self):

        noise = self.interferometers.frequency_domain_strain
        psd = self.interferometers.power_spectral_density_array
        duration = self.waveform_generator.duration

        log_l = -2.0 / duration * np.sum(np.abs(noise)**2 / psd)
        
        return log_l.real

In [ ]:
#### Define the sampling frequency and the data duration
sampling_frequency = H1.sampling_frequency
duration = H1.duration


#### Call the waveform_generator to create our waveform model.
waveform = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=echo,
)

In [ ]:
#### define parameters to inject.
injection_parameters = dict(
    R=0.5,
    d=100,
    M=63,
    D=0.4,
    wr=0.3737,
    wi=-0.08896,
    alpha=0.2161,
    t_0=-0.001,
    phi=0.0,
    ra=2.19432,
    dec=-1.2232,
    psi=0.532268,
    geocent_time=1126259462.4
)


#### Inject the signal
H1.inject_signal(
    waveform_generator=waveform, parameters=injection_parameters, raise_error=False
)

In [ ]:
#### Define the prior for our parameters
prior = injection_parameters.copy()
prior['R'] = bilby.core.prior.Uniform(name='Reflection', minimum=0.3,maximum=0.8)
prior['d'] = bilby.core.prior.Uniform(name='Compactness', minimum=80, maximum=120)
prior['M'] = bilby.core.prior.Uniform(name="Mass", minimum=40, maximum=80, unit="$M_{sun}$")
prior['D'] = bilby.core.prior.Uniform(name="Luminosity Distance", minimum=0.2, maximum=0.6, unit="$Gpc$")

In [ ]:
#### Instantiate the Likelihood
likelihood = My_Likelihood(interferometers=[H1], waveform_generator=waveform, priors=prior)

In [ ]:
#### launch sampler
result = bilby.core.sampler.run_sampler(
    likelihood,
    prior,
    sampler="dynesty",
    npoints=500,
    walks=5,
    nact=3,
    injection_parameters=injection_parameters,
    outdir="Inject_recover",
    label="Echo_recover",
    dlogz=0.01
)

#### This will automatically show the signal-to-noise Bayes factor
#### Plot the corner plot
result.plot_corner()

In [ ]:
#### Plot the recovered signal
idxs = likelihood.interferometers[0].strain_data.frequency_mask  # This is a boolean mask of the frequencies which we'll use in the analysis
plt.figure(figsize=(10, 6))
plt.loglog(likelihood.interferometers[0].frequency_array[idxs],
          likelihood.interferometers[0].amplitude_spectral_density_array[idxs], label='H1 ASD', alpha=0.5, color='blue')
plt.loglog(waveform.frequency_array[idxs], np.sqrt(waveform.frequency_array[idxs])*
           np.abs(waveform.frequency_domain_strain()['plus'][idxs]), label='Recoverd template', color='red')
plt.fill_between(waveform.frequency_array[idxs], np.sqrt(waveform.frequency_array[idxs])*
                 np.abs(waveform.frequency_domain_source_model(waveform.frequency_array, 0.45-0.05, 100.61-1.46, 62.71-0.88, 
                                                               0.37-0.02, 0.3737, -0.08896, 0.2161, 0, 0)['plus'])[idxs], 
                 np.sqrt(waveform.frequency_array[idxs])*
                 np.abs(waveform.frequency_domain_source_model(waveform.frequency_array, 0.45+0.06, 100.61+1.50, 62.71+0.85, 
                                                               0.37+0.02, 0.3737, -0.08896, 0.2161, 0, 0)['plus'])[idxs],
                 alpha=0.6, color='green')

plt.xlabel('Frequency [Hz]')
plt.legend(framealpha=0.6)
plt.ylabel("Strain Amplitude")
plt.title("Best-Fit template")
plt.grid(True, which="both", ls=":")
plt.tight_layout()
#plt.savefig("./Freq_domain_recover_vs_psd.png", dpi=300)
plt.show()

In [ ]:
#### Calculate the SNR
sig = (1/(2*np.pi))*np.sqrt(waveform.frequency_array[idxs])*np.abs(waveform.frequency_domain_strain()['plus'][idxs])
snr = np.sqrt(4/H1.duration * np.sum((likelihood.interferometers[0].frequency_domain_strain[idxs]*np.conj(sig))/
                               likelihood.interferometers[0].power_spectral_density_array[idxs]).real)

print("The SNR:", snr)